<a href="https://colab.research.google.com/github/prp1519/prp1519/blob/main/modeltorunall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq transformers sentencepiece datasets

In [ ]:
!pip install -qq pytorch-lightning

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [ ]:
from datasets import load_dataset

train_df = load_dataset("json", data_files="test.json")

Using custom data configuration default-38c1068d2b95da95


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:01<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-38c1068d2b95da95/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset

val_df = load_dataset("json", data_files="dev.json")

Using custom data configuration default-dd376262a6be6018


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-dd376262a6be6018/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
train_df

DatasetDict({
    train: Dataset({
        features: ['uid', 'context', 'hypothesis', 'label', 'model_label', 'emturk', 'genre', 'reason', 'tag'],
        num_rows: 1
    })
})

In [ ]:
from transformers import default_data_collator
data_collator = default_data_collator

In [ ]:
from transformers import TrainingArguments, Trainer
import os 

In [ ]:
!pip install pytorch

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-p1ui_e4n/pytorch_feacfca61e144db18eefe8f396d2b45d/setup.py'"'"'; __file__='"'"'/tmp/pip-install-p1ui_e4n/pytorch_feacfca61e144db18eefe8f396d2b45d/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-ruthqzsz/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/pytorch Check the logs for full command output.


In [ ]:
hg_model_hub_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/albert-xxlarge-v2-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/bart-large-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/electra-large-discriminator-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/xlnet-large-cased-snli_mnli_fever_anli_R1_R2_R3-nli"

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(hg_model_hub_name)
model = AutoModelForSequenceClassification.from_pretrained(hg_model_hub_name)

loading configuration file https://huggingface.co/ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c885540dd6f48d63a0ab21cce2e77c78d7ffa73aaaae4fc7607443195539403c.90c8346d0c934a0eaa987c774c09aa24c0b9fbd0b76e1c1413dc1a165805ec71
Model config RobertaConfig {
  "_name_or_path": "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "entailment",
    "1": "neutral",
    "2": "contradiction"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "contradiction": 2,
    "entailment": 0,
    "neutral": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "mode

In [ ]:
def preprocess_function(examples):
    hypothesis = examples['hypothesis']
    premise = examples['context']
    label = examples['label']
    inputs = tokenizer(
        
        hypothesis,
        premise,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )
    inputs['label'] = label
    return inputs

In [ ]:
tokenized_anli = train_df.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
print(tokenized_anli)

DatasetDict({
    train: Dataset({
        features: ['uid', 'context', 'hypothesis', 'label', 'model_label', 'emturk', 'genre', 'reason', 'tag', 'input_ids', 'attention_mask', 'offset_mapping'],
        num_rows: 1
    })
})


In [ ]:
tokenized_anli1 = val_df.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
training_args = TrainingArguments(
    output_dir='./roberta',
    #trainer = Trainer(default_root_dir="/your/path/to/save/checkpoints"),
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_anli["train"],
    eval_dataset=tokenized_anli1["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: offset_mapping, hypothesis, tag, reason, model_label, context, uid, genre, emturk.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3


Epoch,Training Loss,Validation Loss
1,No log,9.363566
2,No log,10.122452
3,No log,10.183651


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: offset_mapping, hypothesis, tag, reason, model_label, context, uid, genre, emturk.
***** Running Evaluation *****
  Num examples = 1
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: offset_mapping, hypothesis, tag, reason, model_label, context, uid, genre, emturk.
***** Running Evaluation *****
  Num examples = 1
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: offset_mapping, hypothesis, tag, reason, model_label, context, uid, genre, emturk.
***** Running Evaluation *****
  Num examples = 1
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3, training_loss=0.01537452389796575, metrics={'train_runtime': 74.2613, 'train_samples_per_second': 0.04, 'train_steps_per_second': 0.04, 'total_flos': 2096852652288.0, 'train_loss': 0.01537452389796575, 'epoch': 3.0})

In [ ]:
predictions = trainer.predict(tokenized_anli1["train"])
print(predictions.predictions.shape, predictions.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: offset_mapping, hypothesis, tag, reason, model_label, context, uid, genre, emturk.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8


(1, 3) (1,)


In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
from datasets import load_metric

m1 = load_metric("f1")
z1=m1.compute(predictions=preds, references=predictions.label_ids)
print(z1)
m2 = load_metric("recall")
z2=m2.compute(predictions=preds, references=predictions.label_ids)
print(z2)
m3 = load_metric("accuracy")
z3=m3.compute(predictions=preds, references=predictions.label_ids)
print(z3)
m4 = load_metric("matthews_correlation")
z4=m4.compute(predictions=preds, references=predictions.label_ids)
print(z4)
#m5 = load_metric("pearsonr")
#z5=m5.compute(predictions=preds, references=predictions.label_ids)
#print(z5)
m6 = load_metric("precision")
z6=m6.compute(predictions=preds, references=predictions.label_ids)
print(z6)
m7 = load_metric("spearmanr")
z7=m7.compute(predictions=preds, references=predictions.label_ids)
print(z7)
m8 = load_metric("xnli")
z8=m8.compute(predictions=preds, references=predictions.label_ids)
print(z8)

{'f1': 0.0}
{'recall': 0.0}
{'accuracy': 0.0}
{'matthews_correlation': 0.0}
{'precision': 0.0}


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'spearmanr': nan}
{'accuracy': 0.0}
